# Tarea 15: Pipelines ETL, DataOps y Orquestación con Prefect




## Parte 1 — Investigación: Conceptos Fundamentales de Prefect (15 min)

Antes de escribir código, investigamos la documentación oficial de Prefect y respondemos las siguientes preguntas. **Deben incluirse citas o referencias específicas de la documentación.**


### 1.1 Tasks en Prefect

**Documentación oficial:** [Prefect Tasks](https://docs.prefect.io/latest/concepts/tasks/)

#### 1. ¿Qué es una Task en Prefect?

**Respuesta:**

Una **Task** en Prefect es una unidad de trabajo individual que representa una operación específica dentro de un pipeline. Según la [documentación oficial de Prefect](https://docs.prefect.io/latest/concepts/tasks/), una task es "a discrete piece of work that can be tracked and retried independently". 

Las tasks son funciones Python decoradas con `@task` que encapsulan una lógica de trabajo específica (por ejemplo, extraer datos, transformar un DataFrame, cargar en una base de datos). Son la unidad básica de ejecución en Prefect y pueden tener dependencias entre ellas, lo que permite que Prefect construya automáticamente un DAG (grafo acíclico dirigido) de ejecución.

**Cuándo usarla:** Se usa cuando necesitas dividir un flujo de trabajo en pasos discretos, reutilizables y rastreables. Cada task debe realizar una acción específica y bien definida.


#### 2. ¿Qué significa que las Tasks sean "lazily evaluated"?

**Respuesta:**

La evaluación diferida ("lazy evaluation") significa que las tasks **no se ejecutan inmediatamente** cuando se definen o cuando se llama a la función decorada. En su lugar, Prefect construye una representación del grafo de dependencias primero.

Cuando llamas a una task dentro de un flow, Prefect no ejecuta el código de la task de inmediato. En lugar de eso, registra la llamada y las dependencias. La ejecución real ocurre cuando se ejecuta el flow completo, momento en el cual Prefect resuelve las dependencias y ejecuta las tasks en el orden correcto.

Esto permite que Prefect optimice la ejecución, maneje dependencias complejas, y construya el DAG antes de ejecutar cualquier cosa. Según la [documentación de Prefect](https://docs.prefect.io/latest/concepts/flows/), esto es parte del concepto de "imperative orchestration" donde defines el flujo de manera imperativa pero Prefect lo ejecuta de manera declarativa.


#### 3. ¿Qué son los Task States?

**Respuesta:**

Los **Task States** representan el estado actual de una task durante su ciclo de vida. Según la [documentación de Prefect sobre States](https://docs.prefect.io/latest/concepts/states/), algunos estados posibles son:

| Estado | ¿Cuándo ocurre? |
|--------|----------------|
| **PENDING** | La task está esperando ser ejecutada. Estado inicial cuando se crea la task y está en cola. |
| **RUNNING** | La task está actualmente en ejecución. Ocurre cuando el código de la task está siendo procesado activamente. |
| **COMPLETED** | La task se ejecutó exitosamente y retornó un resultado. Ocurre cuando la ejecución termina sin errores. |
| **FAILED** | La task falló durante su ejecución debido a una excepción. Ocurre cuando se lanza una excepción no capturada. |
| **RETRYING** | La task está siendo reintentada después de un fallo. Ocurre cuando se configura `retries` y la task falla, antes del siguiente intento. |
| **CANCELLED** | La task fue cancelada antes de completarse. Ocurre cuando el flow es cancelado o interrumpido externamente. |
| **CRASHED** | La task falló de manera inesperada o no manejada. Estado de fallo crítico cuando no hay manejo de errores. |


#### 4. ¿Qué parámetros importantes tiene el decorador `@task`?

**Respuesta:**


Según la [documentación de Prefect Tasks](https://docs.prefect.io/latest/api-ref/prefect/tasks/), algunos parámetros importantes son:

| Parámetro | ¿Qué hace? | Ejemplo de uso |
|-----------|------------|----------------|
| **`retries`** | Define cuántas veces reintentar la task si falla. Por defecto es 0. | `@task(retries=3)` - La task intentará hasta 3 veces antes de fallar definitivamente. |
| **`retry_delay_seconds`** | Tiempo de espera entre reintentos en segundos. Útil para tareas que pueden fallar temporalmente. | `@task(retries=2, retry_delay_seconds=5)` - Espera 5 segundos entre reintentos. |
| **`timeout_seconds`** | Tiempo máximo de ejecución para la task. Si excede, se cancela automáticamente. | `@task(timeout_seconds=300)` - La task debe completarse en 5 minutos o se cancela. |
| **`cache_key_fn`** | Función personalizada para generar la clave de caché. Permite controlar cuándo se cachea un resultado. | `@task(cache_key_fn=lambda task, context: context['parameters']['date'])` - Cachea basado en un parámetro específico. |
| **`cache_expiration`** | Duración para la cual el resultado cacheado es válido. Puede ser un timedelta. | `@task(cache_expiration=timedelta(hours=1))` - El cache expira después de 1 hora. |
| **`tags`** | Etiquetas para organizar y filtrar tasks. Útil para monitoreo y organización. | `@task(tags=["extract", "production"])` - Etiqueta la task para filtrado y organización. |
| **`log_prints`** | Si es True, captura los prints como logs de Prefect. Facilita el debugging. | `@task(log_prints=True)` - Los prints se registran automáticamente en los logs de Prefect. |


### 1.2 Flows en Prefect

**Documentación oficial:** [Prefect Flows](https://docs.prefect.io/latest/concepts/flows/)

#### 1. ¿Cuál es la diferencia entre un Flow y una Task? ¿Por qué necesitamos ambos?

**Respuesta:**

Según la [documentación de Prefect](https://docs.prefect.io/latest/concepts/flows/), un **Flow** es un contenedor que orquesta y coordina la ejecución de múltiples Tasks. Mientras que una Task representa una unidad de trabajo individual, un Flow representa el flujo de trabajo completo que conecta múltiples Tasks.

**Diferencias clave:**
- **Task**: Unidad básica de trabajo, realiza una acción específica
- **Flow**: Orquesta múltiples tasks, define las dependencias y el orden de ejecución

**¿Por qué necesitamos ambos?**
- Las Tasks nos permiten dividir el trabajo en componentes reutilizables y rastreables
- Los Flows nos permiten coordinar las Tasks, manejar dependencias automáticamente, y proporcionar un contexto de ejecución unificado. Sin Flows, las Tasks serían funciones aisladas sin orquestación.


#### 2. ¿Qué es un "subflow"? ¿Cuándo sería útil usar subflows?

**Respuesta:**

Un **subflow** es un Flow que se puede llamar desde dentro de otro Flow. Según la [documentación de Prefect sobre subflows](https://docs.prefect.io/latest/concepts/flows/#composing-flows), cuando llamas a un flow decorado con `@flow` desde dentro de otro flow, automáticamente se convierte en un subflow.

**Cuándo sería útil:**
- **Modularidad**: Dividir un pipeline grande en módulos más pequeños y manejables
- **Reutilización**: Reutilizar flujos comunes en múltiples pipelines principales
- **Organización**: Organizar lógicamente grupos relacionados de tasks
- **Debugging**: Facilitar el debugging al aislar secciones del pipeline
- **Testing**: Probar secciones específicas del pipeline de manera independiente

Por ejemplo, podrías tener un subflow `extract_and_validate()` que se use en múltiples pipelines ETL diferentes.


#### 3. ¿Cómo maneja Prefect las dependencias entre tasks? Expliquen el concepto de DAG implícito.

**Respuesta:**

Prefect maneja las dependencias entre tasks de manera **automática e implícita** a través de lo que se conoce como "DAG implícito". Según la [documentación de Prefect](https://docs.prefect.io/latest/concepts/flows/#task-dependencies), Prefect detecta automáticamente las dependencias basándose en cómo se llaman las tasks dentro de un flow.

**Cómo funciona:**
1. Cuando llamas a una task dentro de un flow y pasas el resultado de otra task como parámetro, Prefect automáticamente detecta esa dependencia
2. Prefect construye un DAG (grafo acíclico dirigido) basándose en estas dependencias implícitas
3. Las tasks se ejecutan en el orden correcto según sus dependencias, no necesariamente en el orden en que aparecen en el código

**Ejemplo:**
```python
@flow
def my_flow():
    data = extract_data()  # Task 1
    transformed = transform_data(data)  # Task 2 depende de Task 1
    load_data(transformed)  # Task 3 depende de Task 2
```

Prefect detecta automáticamente que `transform_data` depende de `extract_data`, y `load_data` depende de `transform_data`, construyendo un DAG sin necesidad de especificarlo explícitamente.


### 1.3 Investigación avanzada: Results y Caching

**Documentación oficial:** [Prefect Results](https://docs.prefect.io/latest/concepts/results/) y [Caching](https://docs.prefect.io/latest/concepts/tasks/#caching)

#### 1. ¿Qué es el "result persistence"? ¿Por qué es importante en pipelines de datos?

**Respuesta:**

El **result persistence** se refiere a la capacidad de Prefect de almacenar y recuperar los resultados de las tasks. Según la [documentación de Prefect sobre Results](https://docs.prefect.io/latest/concepts/results/), Prefect puede persistir los resultados de las tasks en diferentes backends (local, S3, GCS, etc.).

**¿Por qué es importante en pipelines de datos?**
- **Resiliencia**: Si un flow falla a mitad de camino, no necesitas re-ejecutar todas las tasks desde el inicio
- **Debugging**: Puedes inspeccionar los resultados intermedios para entender qué salió mal
- **Reproducibilidad**: Puedes reproducir exactamente el mismo estado de ejecución
- **Eficiencia**: En caso de fallo, solo necesitas re-ejecutar desde el punto de fallo
- **Auditoría**: Permite rastrear y validar los resultados de cada etapa del pipeline


#### 2. ¿Cómo funciona el caching en Prefect? ¿Qué parámetro usarían para cachear el resultado de una task?

**Respuesta:**

El **caching** en Prefect permite que las tasks eviten la re-ejecución si ya se ejecutaron previamente con los mismos parámetros. Según la [documentación de Prefect sobre Caching](https://docs.prefect.io/latest/concepts/tasks/#caching), Prefect genera automáticamente una clave de caché basándose en los parámetros de entrada de la task.

**Parámetros para cachear:**
- **`cache_key_fn`**: Función personalizada para generar la clave de caché
- **`cache_expiration`**: Duración de validez del caché (ej: `timedelta(hours=1)`)
- **`refresh_cache`**: Forzar la invalidación del caché

**Ejemplo:**
```python
@task(cache_expiration=timedelta(hours=1))
def extract_data(date: str):
    # Esta task solo se ejecutará una vez por hora para la misma fecha
    return fetch_data(date)
```

Si la misma task se llama con los mismos parámetros dentro de la ventana de expiración, Prefect retornará el resultado cacheado en lugar de re-ejecutar la task.


#### 3. ¿Qué es una `cache_key_fn`? Den un ejemplo de cuándo la usarían.

**Respuesta:**

Una **`cache_key_fn`** es una función personalizada que se usa para generar la clave de caché de una task. Según la [documentación de Prefect](https://docs.prefect.io/latest/concepts/tasks/#cache-key-functions), permite controlar exactamente qué factores determinan si una task debe ser re-ejecutada o usar el caché.

**Ejemplo de uso:**

```python
@task(
    cache_key_fn=lambda task, context: f"{context['parameters']['date']}_{context['parameters']['region']}"
)
def extract_regional_data(date: str, region: str, api_key: str):
    # Esta task se cachea basándose en date y region, pero NO en api_key
    return fetch_data(date, region, api_key)
```

**Cuándo usarla:**
- Cuando solo quieres cachear basándote en **algunos parámetros** (no todos)
- Cuando necesitas incluir factores externos (como la hora del día, estado de archivos) en la clave de caché
- Cuando quieres invalidar el caché basándote en lógica de negocio compleja
- Por ejemplo: cachear datos de ventas solo basándote en la fecha, ignorando otros parámetros como IDs de usuario que no afectan el resultado


## Parte 2 — Diseño Conceptual (5 min)

Definimos un **escenario simple** para nuestro pipeline.


### 2.1 Arquitectura del escenario

**Escenario elegido**: Ventas de un e-commerce con datos de transacciones diarias

| Rol | ¿Quién sería en nuestro escenario? |
|-----|-----------------------------------|
| **Business data owner** | Equipo de ventas/negocios que genera las transacciones diarias en la plataforma e-commerce |
| **Data engineers** | Equipo de ingeniería de datos que construye y mantiene el pipeline ETL para procesar y cargar las ventas |
| **Data consumers** | Analistas de negocio, científicos de datos y dashboards que consumen los datos procesados para análisis y reportes |


### 2.2 Tipo de pipeline

* **Tipo elegido**: Batch
* **Justificación**: Las ventas se procesan diariamente en lotes. No necesitamos procesamiento en tiempo real para este caso de uso inicial. El procesamiento batch es más eficiente para análisis agregados y permite procesar grandes volúmenes de datos de manera controlada. Además, facilita la validación y el manejo de errores antes de cargar los datos procesados.


## Parte 3 — Implementación del Pipeline Base (20 min)


### 3.1 Setup

In [1]:
# Instalación de Prefect
# !pip install -q prefect pandas

# Importar librerías
from prefect import flow, task
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("✅ Entorno configurado correctamente")
print(f"📅 Fecha: {datetime.now().strftime('%Y-%m-%d %H:%M')}")


ModuleNotFoundError: No module named 'prefect'

### 3.2 Implementar Tasks

Basándose en lo que investigamos en la Parte 1, implementamos las tasks con los decoradores y parámetros correctos.


In [ ]:
# === TASK 1: EXTRACT ===
# Decorador @task con parámetros útiles: tags para organización y log_prints para capturar prints
@task(tags=["extract", "data-source"], log_prints=True)
def extract_data():
    """
    Extrae datos de la fuente.
    Simula la extracción de datos de ventas de un e-commerce.
    """
    np.random.seed(42)
    n_rows = 100

    data = {
        'fecha': pd.date_range(start='2024-01-01', periods=n_rows, freq='D'),
        'producto': np.random.choice(['A', 'B', 'C', 'D'], n_rows),
        'cantidad': np.random.randint(1, 50, n_rows),
        'precio_unitario': np.random.uniform(10, 100, n_rows).round(2),
        'region': np.random.choice(['Norte', 'Sur', 'Este', 'Oeste'], n_rows)
    }

    df = pd.DataFrame(data)
    print(f"📥 Extraídos {len(df)} registros")
    return df


In [ ]:
# === TASK 2: TRANSFORM ===
# Decorador @task con tags para organización
@task(tags=["transform", "data-processing"], log_prints=True)
def transform_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Aplica transformaciones a los datos.
    Calcula totales, categorías de ticket size, y limpia los datos.
    """
    # Calcular total
    df['total'] = df['cantidad'] * df['precio_unitario']
    
    # Categorizar por ticket size
    df['ticket_size'] = pd.cut(
        df['total'],
        bins=[0, 100, 500, 1000, float('inf')],
        labels=['Bajo', 'Medio', 'Alto', 'Muy Alto']
    )
    
    # Agregar mes y día de la semana para análisis
    df['mes'] = df['fecha'].dt.month
    df['dia_semana'] = df['fecha'].dt.day_name()
    
    print(f"🔄 Transformados {len(df)} registros")
    print(f"   Total vendido: ${df['total'].sum():,.2f}")
    
    return df


In [ ]:
# === TASK 3: LOAD ===
# Decorador @task con tags y retries por si falla la escritura
@task(tags=["load", "data-output"], log_prints=True, retries=2, retry_delay_seconds=3)
def load_data(df: pd.DataFrame, output_path: str = "ventas_procesadas.csv") -> str:
    """
    Carga los datos transformados en el destino.
    En producción, esto podría ser una base de datos, data warehouse, etc.
    """
    df.to_csv(output_path, index=False)
    print(f"💾 Datos cargados en: {output_path}")
    print(f"   Registros guardados: {len(df)}")
    return output_path


### 3.3 Implementar Flow

Ahora creamos el flow principal que orquesta las tasks.


In [ ]:
# === FLOW PRINCIPAL ===
@flow(name="ETL Pipeline Ventas", log_prints=True)
def etl_flow():
    """
    Flow principal que orquesta el pipeline ETL completo.
    Prefect detecta automáticamente las dependencias entre tasks:
    - transform_data depende de extract_data
    - load_data depende de transform_data
    """
    print("🚀 Iniciando pipeline ETL...")
    
    # Paso 1: Extraer datos
    df_raw = extract_data()
    
    # Paso 2: Transformar datos (depende de extract_data)
    df_transformed = transform_data(df_raw)
    
    # Paso 3: Cargar datos (depende de transform_data)
    output_file = load_data(df_transformed)
    
    print(f"✅ Pipeline completado. Archivo generado: {output_file}")
    return output_file


### 3.4 Ejecutar el Flow

Ejecutamos el flow para ver cómo funciona Prefect:


In [ ]:
# Ejecutar el flow
if __name__ == "__main__":
    result = etl_flow()
    print(f"\n🎉 Flow ejecutado exitosamente. Resultado: {result}")


In [ ]:
# Ejecutar el flow (en notebook)
result = etl_flow()
print(f"\n🎉 Flow ejecutado exitosamente. Resultado: {result}")


### 3.4 Preguntas de observación

#### 1. ¿Qué observan en los logs de Prefect?
**Respuesta:** Los logs muestran el estado de cada task (PENDING → RUNNING → COMPLETED), los mensajes de print capturados por `log_prints=True`, y un resumen del flujo de ejecución. Prefect proporciona información detallada sobre el tiempo de ejecución de cada task y el flujo completo.

#### 2. ¿Cómo se construyó el DAG?
**Respuesta:** Prefect construyó el DAG automáticamente basándose en las dependencias implícitas detectadas cuando pasamos el resultado de una task como parámetro de otra. El DAG es: `extract_data` → `transform_data` → `load_data`.

#### 3. ¿Qué pasaría si una task falla?
**Respuesta:** Si una task falla, Prefect marca el estado como FAILED y detiene la ejecución del flow (a menos que configuremos manejo de errores). Las tasks dependientes no se ejecutarán. Con `retries` configurado, Prefect reintentará la task antes de marcar como fallida.


## Parte 4 — Investigación: Funcionalidades Avanzadas (15 min)


### 4.1 Retries y manejo de errores

**Investigación:** [Prefect Retries](https://docs.prefect.io/latest/concepts/tasks/#retries)

**Respuestas:**

1. **¿Cómo funcionan los retries?** Prefect permite configurar reintentos automáticos cuando una task falla. Se configuran con `@task(retries=N, retry_delay_seconds=X)` donde N es el número de reintentos y X es el tiempo de espera entre intentos.

2. **¿Cuándo usar retries?** Útil para operaciones que pueden fallar temporalmente (APIs, conexiones de red, operaciones de I/O). No es recomendable para errores de lógica que siempre fallarán.

3. **Ya lo implementamos:** En la task `load_data()` configuramos `retries=2, retry_delay_seconds=3` para manejar posibles fallos en la escritura del archivo.


### 4.2 Caching de resultados

**Investigación:** [Prefect Caching](https://docs.prefect.io/latest/concepts/tasks/#caching)

**Respuestas:**

1. **¿Cómo funciona el caching?** Prefect genera automáticamente una clave de caché basada en los parámetros de entrada. Si la misma task se ejecuta con los mismos parámetros, retorna el resultado cacheado en lugar de re-ejecutar.

2. **Parámetros clave:** `cache_expiration` (timedelta) y `cache_key_fn` (función personalizada).

3. **Cuándo usar:** Para tasks costosas que pueden reutilizar resultados (extracciones de APIs, procesamiento pesado) cuando los parámetros no cambian.


### 4.3 Logging personalizado

**Investigación:** [Prefect Logging](https://docs.prefect.io/latest/concepts/logs/)

**Respuestas:**

1. **¿Cómo usar logging?** Se puede usar `get_run_logger()` dentro de una task para obtener un logger estructurado, o configurar `log_prints=True` para capturar automáticamente los prints.

2. **Ya lo implementamos:** Usamos `log_prints=True` en todas nuestras tasks para que los prints se registren como logs estructurados de Prefect.

3. **Ventajas:** Logs centralizados, niveles de log (INFO, WARNING, ERROR), y integración con la UI de Prefect para debugging.


### 4.4 Concurrencia y paralelismo

**Investigación:** [Prefect Task Runners](https://docs.prefect.io/latest/concepts/task-runners/)

**Respuestas:**

1. **¿Cómo funciona?** Prefect soporta ejecución concurrente usando `ConcurrentTaskRunner()` como parámetro del flow. Las tasks independientes pueden ejecutarse en paralelo.

2. **Cuándo usar:** Cuando tienes múltiples tasks que no dependen una de otra y quieres reducir el tiempo total de ejecución.

3. **Ejemplo:** Si procesamos datos por región, podemos usar `futures = [process_region.submit(r) for r in regions]` para ejecutar en paralelo.


## Parte 5 — Investigación: Deployments y Scheduling (10 min)

**Documentación:** [Deployments](https://docs.prefect.io/latest/concepts/deployments/) y [Schedules](https://docs.prefect.io/latest/concepts/schedules/)


### 5.1 Conceptos de Deployment

**Respuestas basadas en la documentación:**

1. **¿Qué es un Deployment en Prefect?** Un Deployment es una configuración que permite ejecutar un Flow de manera programada o bajo demanda. Según la [documentación](https://docs.prefect.io/latest/concepts/deployments/), un Deployment conecta un Flow con una configuración de ejecución específica (scheduling, work pool, parámetros). La diferencia con un Flow es que un Flow es el código, mientras que un Deployment es la "instalación" del Flow en Prefect Cloud/Server para ejecutarlo.

2. **¿Qué es un Work Pool?** Un Work Pool es un grupo de workers que pueden ejecutar flows. Según la [documentación](https://docs.prefect.io/latest/concepts/work-pools/), permite organizar dónde y cómo se ejecutan los flows (local, servidor, cloud). Es una abstracción para manejar la infraestructura de ejecución.

3. **¿Qué es un Worker?** Un Worker es un proceso que ejecuta flows desde un Work Pool. Los Workers se conectan a un Work Pool y "toman" trabajos (flows) para ejecutarlos. La relación es: Deployment → Work Pool → Worker → Ejecución del Flow.


### 5.2 Scheduling

**Tipos de schedules soportados por Prefect:**

| Tipo de Schedule | Descripción | Ejemplo |
|-----------------|-------------|---------|
| **CronSchedule** | Usa sintaxis cron para definir horarios recurrentes | `cron="0 6 * * *"` - Todos los días a las 6 AM |
| **IntervalSchedule** | Ejecuta a intervalos fijos de tiempo | `interval=timedelta(hours=1)` - Cada hora |
| **RRuleSchedule** | Usa reglas RFC 5545 (iCalendar) para horarios complejos | Útil para horarios de negocio, excluir fines de semana |
| **Manual** | Sin schedule, solo ejecución manual | Para ejecución bajo demanda |

**1. ¿Cómo expresarían "ejecutar todos los días a las 6 AM" en cron?**
**Respuesta:** `"0 6 * * *"` - El formato cron es: minuto hora día_mes mes día_semana

**2. ¿Qué es `RRuleSchedule`?** 
**Respuesta:** RRuleSchedule permite definir horarios complejos usando reglas RFC 5545, como "cada lunes y miércoles a las 9 AM, excepto feriados" o "último día del mes". Es más flexible que cron para casos de negocio complejos.


### 5.3 Crear un Deployment (conceptual)

Basándose en la documentación, código para crear un deployment del flow:


In [ ]:
# TODO: Código conceptual para crear un deployment
# https://docs.prefect.io/latest/concepts/deployments/

from prefect import flow
from prefect.schedules import CronSchedule

# Opción 1: Usando serve() - más simple para desarrollo
if __name__ == "__main__":
    etl_flow.serve(
        name="etl-ventas-daily",  # nombre del deployment
        cron="0 6 * * *",  # schedule: todos los días a las 6 AM
        tags=["production", "etl", "ventas"],  # tags para organización
    )

# Opción 2: Usando deploy() - más control para producción
# etl_flow.deploy(
#     name="etl-ventas-daily",
#     work_pool_name="default",  # o un work pool específico
#     cron="0 6 * * *",
#     tags=["production", "etl"],
# )


## Parte 6 — Extensión DataOps (15 min)

Elegimos implementar la **Opción A — Validación con logging estructurado**, ya que es fundamental para la calidad de datos en pipelines ETL.


In [ ]:
# === OPCIÓN A: VALIDACIÓN CON LOGGING ESTRUCTURADO ===
from prefect import get_run_logger

@task(retries=1, retry_delay_seconds=2, tags=["validation", "data-quality"], log_prints=True)
def validate_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Valida la calidad de los datos usando logging estructurado de Prefect.
    Esta task valida que los datos cumplan con las expectativas antes de continuar.
    
    Basado en: https://docs.prefect.io/latest/concepts/logs/
    """
    logger = get_run_logger()  # Obtiene el logger estructurado de Prefect
    errors = []

    logger.info("Iniciando validación de datos")
    logger.info(f"DataFrame recibido con {len(df)} registros y {len(df.columns)} columnas")

    # Validación 1: DataFrame no vacío
    if len(df) <= 0:
        logger.error("DataFrame vacío detectado - No se pueden procesar datos vacíos")
        errors.append("DataFrame vacío")
    else:
        logger.info(f"✅ Validación de cantidad de registros: OK ({len(df)} registros)")

    # Validación 2: Valores nulos
    null_counts = df.isnull().sum()
    if null_counts.sum() > 0:
        logger.warning(f"Valores nulos encontrados: {null_counts[null_counts > 0].to_dict()}")
        # No agregamos a errors si hay nulos, solo lo registramos como warning
        # En producción, podrías decidir si esto es crítico o no
    else:
        logger.info("✅ Validación de valores nulos: OK (sin nulos)")

    # Validación 3: Columnas requeridas
    required_columns = ['fecha', 'producto', 'cantidad', 'precio_unitario', 'total']
    missing_columns = [col for col in required_columns if col not in df.columns]
    if missing_columns:
        logger.error(f"Columnas requeridas faltantes: {missing_columns}")
        errors.append(f"Columnas faltantes: {missing_columns}")
    else:
        logger.info(f"✅ Validación de columnas requeridas: OK")

    # Validación 4: Tipos de datos básicos
    if 'total' in df.columns:
        if df['total'].dtype not in [float, 'float64', 'float32', int, 'int64', 'int32']:
            logger.error(f"Tipo de dato incorrecto para 'total': {df['total'].dtype}")
            errors.append("Tipo de dato incorrecto en columna 'total'")
        else:
            logger.info("✅ Validación de tipos de datos: OK")

    # Validación 5: Valores negativos en campos que no deberían tenerlos
    if 'cantidad' in df.columns:
        negative_qty = (df['cantidad'] < 0).sum()
        if negative_qty > 0:
            logger.warning(f"Se encontraron {negative_qty} registros con cantidad negativa")
            # Podrías decidir si esto es un error crítico o solo un warning

    if errors:
        logger.error(f"Validación fallida con {len(errors)} error(es) crítico(s)")
        raise ValueError(f"Validación fallida: {errors}")

    logger.info("✅ Validación exitosa - Todos los checks pasaron")
    return df


Ahora actualizamos el flow para incluir la validación:


In [ ]:
# === FLOW ACTUALIZADO CON VALIDACIÓN ===
@flow(name="ETL Pipeline Ventas con Validación", log_prints=True)
def etl_flow_with_validation():
    """
    Flow principal que orquesta el pipeline ETL completo con validación de calidad.
    Incluye validación entre extract y transform para asegurar calidad de datos.
    """
    print("🚀 Iniciando pipeline ETL con validación...")
    
    # Paso 1: Extraer datos
    df_raw = extract_data()
    
    # Paso 2: Validar datos (EXTENSIÓN DATAOPS)
    # Esta validación asegura que los datos cumplen con las expectativas antes de transformar
    df_validated = validate_data(df_raw)
    
    # Paso 3: Transformar datos (solo si la validación pasa)
    df_transformed = transform_data(df_validated)
    
    # Paso 4: Cargar datos
    output_file = load_data(df_transformed)
    
    print(f"✅ Pipeline completado con validación. Archivo generado: {output_file}")
    return output_file


In [ ]:
# Ejecutar el flow con validación
result = etl_flow_with_validation()
print(f"\n🎉 Flow con validación ejecutado exitosamente. Resultado: {result}")


## Parte 7 — Reflexión y Conexión con DataOps (5 min)


### 7.1 Conceptos de Prefect

**1. ¿Cómo ayuda Prefect a implementar el principio de "Observabilidad" de DataOps?**

**Respuesta:** Prefect implementa observabilidad de múltiples formas:
- **Logging estructurado**: Los logs de Prefect capturan automáticamente el estado, tiempo de ejecución y mensajes de cada task
- **Estados de ejecución**: Cada task tiene estados claros (PENDING, RUNNING, COMPLETED, FAILED) que se pueden monitorear
- **UI centralizada**: Prefect Cloud/Server proporciona una interfaz visual para monitorear todos los flows y tasks en tiempo real
- **Métricas**: Prefect rastrea métricas como tiempo de ejecución, tasa de éxito, y frecuencias de fallo
- **Notificaciones**: Permite configurar alertas cuando los flows fallan

Esto permite que los equipos de datos tengan visibilidad completa sobre sus pipelines, facilitando el debugging y la detección temprana de problemas.

**2. ¿Cómo ayuda el caching a la "Reproducibilidad"?**

**Respuesta:** El caching en Prefect ayuda a la reproducibilidad de varias formas:
- **Resultados consistentes**: Al cachear resultados basados en parámetros, aseguras que las mismas entradas produzcan los mismos resultados
- **Traza de ejecuciones**: Prefect mantiene un historial de ejecuciones cacheadas, permitiendo reproducir estados anteriores
- **Puntos de recuperación**: Si un pipeline falla, puedes reutilizar resultados cacheados de tasks exitosas anteriores
- **Validación**: Permite comparar resultados actuales con resultados cacheados para detectar cambios inesperados

Esto es fundamental en DataOps donde necesitas garantizar que los pipelines produzcan resultados consistentes y reproducibles.

**3. ¿Cómo conectan los Deployments con "CI/CD para datos"?**

**Respuesta:** Los Deployments en Prefect son el equivalente a "releases" en CI/CD tradicional:
- **Versionado**: Los deployments permiten versionar y desplegar diferentes versiones de flows
- **Automatización**: Los schedules permiten automatizar la ejecución de pipelines, similar a pipelines de CI/CD
- **Ambientes**: Puedes tener diferentes deployments para desarrollo, staging y producción
- **Rollback**: Puedes cambiar qué versión de un flow está activa, permitiendo rollback rápido
- **Integración con CI/CD**: Prefect puede integrarse con sistemas de CI/CD (GitHub Actions, GitLab CI) para desplegar automáticamente cambios en los flows

Esto permite aplicar prácticas de DevOps a los pipelines de datos, facilitando despliegues seguros y controlados.


### 7.2 Comparación con alternativas

**1. ¿Qué diferencias hay entre Prefect y Apache Airflow? Mencionen al menos 2.**

**Diferencia 1:** **Filosofía de diseño**: Airflow usa DAGs explícitos definidos en Python usando operadores, mientras que Prefect usa "DAGs implícitos" donde las dependencias se detectan automáticamente basándose en cómo se llaman las tasks. Prefect es más "Python puro" y menos verboso.

**Diferencia 2:** **Curva de aprendizaje**: Prefect tiene una curva de aprendizaje más suave porque es más simple de usar - solo decoras funciones con `@task` y `@flow`. Airflow requiere entender conceptos como DAGs, Operators, XComs, etc., lo que lo hace más complejo para empezar.

**Otra diferencia importante:** Prefect está diseñado para ser más moderno y orientado a Python nativo, mientras que Airflow fue diseñado originalmente más como un sistema de workflow basado en DAGs.

**2. ¿Qué es Dagster? ¿En qué se diferencia de Prefect?**

**Respuesta:** Dagster es otra herramienta de orquestación de pipelines de datos moderna, similar a Prefect. Las principales diferencias son:

- **Enfoque en assets**: Dagster está más orientado a "assets" (datos y modelos) como primitivos de primera clase, mientras que Prefect está más orientado a "tasks" y "flows"
- **Integración con dataframes**: Dagster tiene integración más profunda con bibliotecas de datos como pandas, pyspark, etc.
- **Software-defined assets**: Dagster permite definir pipelines basándose en qué datos (assets) necesitas, no solo en qué tareas ejecutar
- **Ecosistema**: Prefect tiene un ecosistema más amplio y comunidad más grande, mientras que Dagster está más enfocado en equipos de datos grandes

Ambas son alternativas modernas a Airflow, pero Dagster tiene un enfoque más "data-centric" mientras que Prefect es más "workflow-centric".


## Conclusiones

En esta tarea aprendimos:

1. **Conceptos fundamentales** de Prefect: Tasks, Flows, DAGs implícitos, estados, y caching
2. **Implementación práctica** de un pipeline ETL completo con Prefect
3. **Funcionalidades avanzadas**: Retries, logging estructurado, validación de datos
4. **Deployments y scheduling**: Cómo desplegar y programar pipelines en producción
5. **Conexión con DataOps**: Cómo Prefect habilita observabilidad, reproducibilidad y CI/CD para datos

Prefect es una herramienta poderosa para orquestar pipelines de datos de manera moderna, con una curva de aprendizaje suave y excelente integración con el ecosistema Python.

---

**Referencias:**
- [Documentación oficial de Prefect](https://docs.prefect.io/)
- [Prefect Concepts Overview](https://docs.prefect.io/latest/concepts/)
- [Prefect GitHub Examples](https://github.com/PrefectHQ/prefect)
